In [1]:
!pip install sagemaker -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 178.8 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.11.0
    Uninstalling platformdirs-3.11.0:
      Successfully uninstalled platformdirs-3.11.0
  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 2.3.0
    Uninstalling omegaconf-2.3.0:
      Successfully uninstalled omegaconf-2.3.0
  Attempting uninstall: mock
    Found existing installation: mock 5.1.0
    Uninstalling mock-5.1.0:
      Successfully uninstalled mock-5.1.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.162
    Uninstalling botocore-1.34.162:
      Successfully uninstalled botocore-1.34.162
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.162
    Uninstalling boto3-1.34.162:
      Successfully uninstalled boto3-1.34.162
  Attempting uninstall: sagema

In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:200: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


[12/08/24 08:34:47] INFO     Created S3 bucket: sagemaker-us-east-2-864981721048                     ]8;id=427587;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=931561;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#723\723]8;;\

sagemaker role arn: arn:aws:iam::864981721048:role/service-role/AmazonSageMaker-ExecutionRole-20241028T234335
sagemaker session region: us-east-2


In [3]:
from sagemaker.huggingface.model import HuggingFaceModel

# Hub model configuration <https://huggingface.co/models>
hub = {
  'HF_MODEL_ID':'distilbert-base-uncased-distilled-squad', # model_id from hf.co/models
  'HF_TASK':'question-answering'                           # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,                                                # configuration for loading model from Hub
   role=role,                                              # IAM role with permissions to create an endpoint
   transformers_version="4.26",                             # Transformers version used
   pytorch_version="1.13",                                  # PyTorch version used
   py_version='py39',                                      # Python version used
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)


[12/08/24 08:38:05] INFO     Creating model with name:                                              ]8;id=590552;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=84570;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-inference-2024-12-08-08-38-05-126                                 

                    INFO     Creating endpoint-config with name                                     ]8;id=866219;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=94865;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#5889\5889]8;;\
                             huggingface-pytorch-inference-2024-12-08-08-38-05-703                                 

[12/08/24 08:38:06] INFO     Creating endpoint with name                                            ]8;id=847321;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=757516;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4711\4711]8;;\
                             huggingface-pytorch-inference-2024-12-08-08-38-05-703                                 

-----!

{'score': 0.9987204670906067, 'start': 68, 'end': 77, 'answer': 'sagemaker'}

{'score': 0.9702979326248169, 'start': 50, 'end': 54, 'answer': 'shop'}

SageMaker endpoint URL: https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/huggingface-pytorch-inference-2024-12-08-08-38-05-703/invocations
